<a href="https://colab.research.google.com/github/SharvaniBaggani/2203A51557_NLP/blob/main/assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]
(i) Build the Transformer Model on above dataset

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

input_ids = tokens['input_ids']

print(f"Tokenized input ids: {input_ids}")

model = GPT2LMHeadModel.from_pretrained('gpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output = model(input_ids.to(device), labels=input_ids.to(device))
loss = output.loss
logits = output.logits

print(f"Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenized input ids: tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,  1310,  2576,
          3706,  2297, 36032, 17233,    13,  1375,  6151,   284,  3187,   607,
         18410,    11,   508,  5615,   287,   262, 16479,    13,  1881,  1110,
            11,   607,  2802,  1965,   607,   284,  1011,   257,  7988,   286,
         39863,   284,   607, 18410,    13,  1550,   607,   835,   832,   262,
         16479,    11,   673,  1138,   257,  1263,  2089, 17481,   508,  2227,
           284,  4483,   607,    13]])


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Loss: 2.5232996940612793


(ii) Train the model using 20, 60, 70 epochs

In [ ]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

def train_model(num_epochs):
    training_args = TrainingArguments(
        output_dir=f"./results_{num_epochs}_epochs",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=1,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=10,
        save_total_limit=2,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
    )

    trainer.train()
    print(f"Training completed for {num_epochs} epochs.")

train_model(20)
train_model(60)
train_model(70)


Step,Training Loss
10,1.405100
20,0.199500


Training completed for 20 epochs.


Step,Training Loss
10,0.129300
20,0.022800
30,0.019800
40,0.028700
50,0.020100
60,0.056400


Training completed for 60 epochs.


Step,Training Loss
10,0.066000
20,0.014200
30,0.012900
40,0.000100
50,0.010400
60,0.029600
70,0.009200


Training completed for 70 epochs.


 (iii) After training, use the model to generate new text by feeding it an initial seed text



In [ ]:
seed_text = "Once upon a time"

input_ids = tokenizer.encode(seed_text, return_tensors="pt")
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=50,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.7
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods.
One day, her mother asked her to take a basket of goodies to her grandmother. On her way to


(iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.



In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

input_ids = tokens['input_ids']

attention_mask = tokens.get('attention_mask', torch.ones_like(input_ids))

print(f"Tokenized input ids: {input_ids}")

model = GPT2LMHeadModel.from_pretrained('gpt2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

output = model(input_ids.to(device), attention_mask=attention_mask.to(device), labels=input_ids.to(device))
loss = output.loss
logits = output.logits

print(f"Loss: {loss.item()}")

model.eval()
generated_text = model.generate(
    input_ids.to(device),
    max_length=100,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    temperature=0.7,
    num_return_sequences=1
)

generated_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)

print(f"Generated Text: {generated_text}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenized input ids: tensor([[ 7454,  2402,   257,   640,    11,   612,   373,   257,  1310,  2576,
          3706,  2297, 36032, 17233,    13,  1375,  6151,   284,  3187,   607,
         18410,    11,   508,  5615,   287,   262, 16479,    13,  1881,  1110,
            11,   607,  2802,  1965,   607,   284,  1011,   257,  7988,   286,
         39863,   284,   607, 18410,    13,  1550,   607,   835,   832,   262,
         16479,    11,   673,  1138,   257,  1263,  2089, 17481,   508,  2227,
           284,  4483,   607,    13]])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loss: 2.5232996940612793
Generated Text: Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. As she was passing the basket, she found out that she was a wolf. She ran to her grandmother's house, and said that she needed help to feed her wolf. The
